In [1]:
from bokeh.io import output_notebook, show
output_notebook()

Loading BokehJS ...

In [393]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from pandas import DataFrame
from pandas_datareader import data
from bokeh.plotting import figure, output_file, show
from bokeh.models.tools import HoverTool
from bokeh.palettes import Spectral4
from bokeh.models import ColumnDataSource
from bokeh.models import LinearAxis, Range1d
from bokeh.layouts import gridplot




class StockMarketAnalysis:
    def __init__(self,stock,units,startD,endD,period):
        self.stock = stock
        self.startD = startD
        self.endD = endD
        self.period = period
        self.units = units


    def CalculateInvestment(self):
        msft = yf.Ticker(self.stock)
        if self.period=="max":
            msft_h = msft.history(period="max")
        else:
            msft_h = msft.history(start=self.startD, end=self.endD)
        
        msft_h1 = msft_h[(msft_h['Stock Splits']>0.0) | (msft_h['Dividends']>0.0) ]
        msft_h1['Stock Splits'].mask(msft_h1['Stock Splits'] == 0.0,1.0,inplace=True)
        
   		# if Dividends are there or not     
        # if msft_h1==0.0:
        #     print("This company do not provide dividends")

        msft_h2 = msft_h1[['Close']]
        msft_h2_ch = msft_h2.pct_change()

        Inv_1 = self.units*msft_h1.Close[0]
        Inv_3 = []
        for ii in range(1,msft_h1.Dividends.shape[0]):

        	if(msft_h1['Stock Splits'][ii] > 0.0):
        		self.units = msft_h1['Stock Splits'][ii]*self.units
        		Inv_1 = self.units*msft_h1.Close[ii]
        	else:
        		self.units = self.units
	
        	Inv_2 = (Inv_1 + (Inv_1*msft_h2_ch.Close[ii]))

        	if(msft_h1['Dividends'][ii] > 0.0):
        		Inv_1 = Inv_2 + Inv_2*msft_h1.Dividends[ii]/100
        	else:
        		Inv_1 = Inv_2

        	Inv_3.append(Inv_1)
        
        Inv_3.insert(0, msft_h1.Close[0])
        msft_h1["Total Returns"] = Inv_3
        print("Total Stocks: ",self.units)


# output to static HTML file
        output_file("lines.html")
        output_file("stocks.html", title="stocks.py example")

        ds = ColumnDataSource(msft_h1)
        TOOLTIPS = "box_select,lasso_select,help"
    # create a new plot with a datetime axis type
        p = figure(plot_width=800, plot_height=350,tools=TOOLTIPS,\
                   x_axis_type="datetime")

        p.add_tools(HoverTool(
            tooltips=[
                ( 'date',   '@Date{%F}'            ),
                ( 'Total Returns',  '$@{Total Returns}{%0.2f}' ), # use @{ } for field names with spaces
                ( 'dividends',  '@{Dividends}{%0.2f}' ),
            ],

            formatters={
                '@Date'    : 'datetime', # use 'datetime' formatter for '@date' field
                '@{Total Returns}' : 'printf',   # use 'printf' formatter for '@{adj close}' field
                                        # use default 'numeral' formatter for other fields
                '@{Dividends}' : 'printf',
            },

            # display a tooltip whenever the cursor is vertically in line with a glyph
        #    mode='vline'
        ))
        p.background_fill_color="#f5f5f5"
        p.grid.grid_line_color="white"
        p.xaxis.axis_label = 'Date'
        p.yaxis.axis_label = 'Price'
        p.axis.axis_line_color = None

        p.extra_y_ranges =  {"Dividends": Range1d(start=0, end=msft_h1.Dividends.max())}
        p.line(y='Total Returns',x='Date',source=ds,color='navy')
        p.line(y='Dividends',x='Date',source=ds,color='green',y_range_name="Dividends")
        p.add_layout(LinearAxis(y_range_name="Dividends", axis_label="Dividends"), 'right')
        p = gridplot([[p]])
        # show the results
        show(p)

In [394]:
msft = yf.Ticker("AAPL")
msft_h1 = msft_h[(msft_h['Stock Splits']>0.0) | (msft_h['Dividends']>0.0) ]

In [395]:
spy = StockMarketAnalysis(stock="AAPL",units=10,startD="2010-01-10",endD="2020-09-11",period="NULL")

In [396]:
spy.CalculateInvestment()

/Users/sopan/opt/miniconda3/envs/default/lib/python3.8/site-packages/pandas/core/generic.py:8992: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._where(
<ipython-input-393-552df4155a9d>:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  msft_h1["Total Returns"] = Inv_3
INFO:bokeh.io.state:Session output file 'lines.html' already exists, will be overwritten.
INFO:bokeh.io.state:Session output file 'stocks.html' already exists, will be overwritten.


Total Stocks:  280.0


In [527]:
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from pandas import DataFrame
from pandas_datareader import data
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
import pandas_bokeh

wordList = ["VOO", "AAPL", "MSFT", "SPY", "SWPPX", "VTI"]
stocks_list = []

for ii in wordList:
    msft = yf.Ticker(ii)
    msft_h = msft.history(start="2010-01-10",end="2020-09-11")
    msft_h1 = msft_h[(msft_h['Stock Splits']>0.0) | (msft_h['Dividends']>0.0) ]
    print(msft_h1.shape)
#    msft_h1['Stocks Name']=ii
    stocks_list.append(msft_h1)
    
stocks_list = pd.concat(stocks_list)    

(41, 7)
(35, 7)
(43, 7)
(42, 7)
(11, 7)
(42, 7)


In [528]:
#stocks_list.plot_bokeh(kind='line',y=['Close','Open'],rangetool=True)
#msft_h2 = pd.MultiIndex.from_frame(msft_h1)
stocks_list.plot_bokeh(kind='line',y=['Close','Open'],rangetool=True)

Column(id='227059', ...)

In [180]:
from bokeh.plotting import figure, output_file, show
from bokeh.models.tools import HoverTool

# output to static HTML file
output_file("lines.html")

window_size = 30
window = np.ones(window_size)/float(window_size)
aapl_avg = np.convolve(msft_h1['Close'], window, 'same')

# output to static HTML file
output_file("stocks.html", title="stocks.py example")

# create a new plot with a datetime axis type
p = figure(plot_width=600, plot_height=350, x_axis_type="datetime")

# add renderers
p.circle(np.array(msft_h1.Close.index, dtype=np.datetime64),  msft_h1['Close'], size=7, color='darkgrey', alpha=0.9, legend_label='close')
p.line(np.array(msft_h1.Close.index, dtype=np.datetime64), aapl_avg, color='navy', legend_label='avg')

# NEW: customize by setting attributes
p.title.text = "AAPL One-Month Average"
p.legend.location = "top_left"
p.grid.grid_line_alpha = 0
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Price'
p.ygrid.band_fill_color = "olive"
p.ygrid.band_fill_alpha = 0.1

# show the results
show(p)


INFO:bokeh.io.state:Session output file 'lines.html' already exists, will be overwritten.
INFO:bokeh.io.state:Session output file 'stocks.html' already exists, will be overwritten.


In [223]:
from bokeh.plotting import figure, output_file, show
from bokeh.models.tools import HoverTool

# output to static HTML file
output_file("lines.html")
output_file("stocks.html", title="stocks.py example")
TOOLTIPS = [
    ("index", "$index"),
    ("(x,y)", "($x, $y)"),
    ("desc", "@desc"),
]


ds = ColumnDataSource(msft_h1)

# create a new plot with a datetime axis type
p = figure(plot_width=600, plot_height=350,tooltips=TOOLTIPS,x_axis_type="datetime")

# add renderers
#p.circle(np.array(aa.index, dtype=np.datetime64),  msft_h1['Close'], size=7, color='darkgrey', alpha=0.9, legend_label='close')
#p.line(np.array(aa.index, dtype=np.datetime64), aapl_avg, color='navy', legend_label='avg')
p.vline_stack(['Close','Open'],x='Date',source=ds,)


# show the results
show(p)



INFO:bokeh.io.state:Session output file 'lines.html' already exists, will be overwritten.
INFO:bokeh.io.state:Session output file 'stocks.html' already exists, will be overwritten.
